In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Normalize

from src.data import EuroSATAllBands
from src.model import Siren
from src.utils import *

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA device")
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
else:
    device = torch.device("cpu")
    print("Using CPU device")

IMAGE_SIZE = 64
DATA_DIR = "data/EuroSATallBands"

Using MPS device


In [2]:
means = [1353.4440, 1115.2527, 1033.7759, 935.1631, 1180.9276, 1966.8143, 2329.8030, 2257.3333,  722.7522,   13.1261, 1781.4470, 1098.1393, 2546.6523]
stds = [63.9742, 150.7762, 183.9667, 272.4519, 223.1591, 348.4093, 445.4806, 519.6234,  96.9965,   1.1972, 370.0003, 296.7262, 491.4442]

transform = Compose([
    Normalize(means, stds)
])

dataset = EuroSATAllBands(root_dir=DATA_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, pin_memory=True, num_workers=0, shuffle=True)

/Users/sviatoslavgladkykh/miniconda/envs/myenv/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
def train_siren(model, model_input, ground_truth):
    # Model Parameters
    total_steps = 10_000
    steps_til_summary = 100
    
    # Early stopping parameters
    patience = 10
    min_delta = 1e-4

    optim = torch.optim.Adam(lr=1e-3, params=model.parameters())

    best_loss = float('inf')
    steps_since_improvement = 0

    for step in range(total_steps):
        model_output, coords = model(model_input)    
        loss = ((model_output - ground_truth) ** 2).mean()
        
        if loss + min_delta < best_loss:
            best_loss = loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
    
        if steps_since_improvement >= patience:
            print(f"Stopping early at step {step} due to no improvement.")
            break
        
        optim.zero_grad()
        loss.backward()
        optim.step()


In [ ]:
all_qualities = []
all_sizes = []

counter = 0
for x, y in dataloader:
    qualities = []
    sizes = []

    for n_hidden in range(2, 101, 2):

        model = Siren(in_features=2,
                      out_features=13,
                      hidden_features=n_hidden,
                      hidden_layers=1,
                      outermost_linear=True).to(device)


        model_input, ground_truth = x.to(device), y.to(device)
        train_siren(model, model_input, ground_truth.to(device))

        compressed = postprocess_model_output(model(model_input)[0])
        real = postprocess_model_output(ground_truth)

        quality = psnr(compressed, real).item()
        bits_per_pixel = bpp(model)
        # n_params = sum(p.numel() for p in model.parameters())

        qualities.append(quality)
        sizes.append(bits_per_pixel)

        show_image(model(model_input)[0].cpu().detach())
        show_image(ground_truth.cpu().detach())

    all_qualities.append(qualities)
    all_sizes.append(sizes)

    counter += 1
    if counter == 1:
        break